## Environment Variables Setup (Standalone)
Set up environment variables for MG5, LHAPDF, Pythia8, and Delphes. Run this cell before any MG5 commands.

In [ ]:
%%bash
# Adjust paths to your installations
export MG5DIR=~/MG5_aMC_v3_6_3
export LHAPDF_DATA_PATH=$MG5DIR/HEPTools/lhapdf6/share/LHAPDF
export LD_LIBRARY_PATH=$MG5DIR/HEPTools/lhapdf6/lib:$LD_LIBRARY_PATH
export PYTHIA8=$MG5DIR/HEPTools/pythia8
export PYTHIA8DATA=$PYTHIA8/share/Pythia8/xmldoc
export LD_LIBRARY_PATH=$PYTHIA8/lib:$LD_LIBRARY_PATH
export DELPHES_DIR=$MG5DIR/Delphes
export LD_LIBRARY_PATH=$DELPHES_DIR:$LD_LIBRARY_PATH
echo "Environment set up."


## Multiparticle Definitions

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
define p = p p~ g
define j = g u c d s u~ c~ d~ s~
define l+ = e+ mu+ ta+
define l- = e- mu- ta-
EOF


### Example: Basic Dilepton Production

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > e+ e-
output example_dilepton
launch
EOF


### Example: Remove Z in Drell–Yan

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > e+ e- / z
output example_dilepton_noZ
launch
EOF


### Example: NLO QCD tt̄ Production

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > t t~ [QCD]
output example_ttbar_NLO
launch
EOF


### Example: MLM Jet Merging jj

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > j j
output example_jj_MLM
launch
set use_syst True
set ickkw 1
set xqcut 30
EOF


### Example: Hadronic W Pair

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > w+ w-, (w+ > j j), (w- > j j)
output example_WW_hadronic
launch
EOF


### Example: Leptonic W Pair

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > w+ w-, (w+ > l+ vl), (w- > l- vl~)
output example_WW_leptonic
launch
EOF


### Example: Semi-Leptonic W Pair

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > w+ w-, (w+ > j j), (w- > l- vl~)
output example_WW_semilepton
launch
EOF


### Example: Mixed tt̄ Decay

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > t t~, (t > b w+), (w+ > j j), (t~ > b~ w-), (w- > l- vl~)
output example_ttbar_mixed
launch
EOF


### Example: MadSpin Spin Correlation

In [ ]:
%%bash
# Prepare MadSpin in standalone
mkdir -p example_ttbar_mixed && cd example_ttbar_mixed
cat > Cards/madspin_card.dat << 'MS'
set spinmode full
define l+ = e+ mu+
define l- = e- mu-
decay t > b l+ vl
decay t~ > b~ l- vl~
launch
MS
$MG5DIR/bin/madspin Cards/madspin_card.dat


### Example: Parameter Reweighting

In [ ]:
%%bash
# Standalone reweight for dilepton
mkdir -p example_dilepton && cd example_dilepton
cat > Cards/reweight_card.dat << 'EOF'
change alpha_EM 0.00729735
change MZ 91.1876
launch
EOF
$MG5DIR/bin/mg5_aMC reweight_card.dat


### Example: Pythia8 Shower WW_leptonic

In [ ]:
%%bash
mkdir -p example_WW_leptonic && cd example_WW_leptonic
echo "set shower=Pythia8" >> Cards/run_card.dat
$MG5DIR/bin/mg5_aMC << 'EOF'
launch
EOF


### Example: Delphes Simulation WW_leptonic

In [ ]:
%%bash
mkdir -p example_WW_leptonic && cd example_WW_leptonic
echo "set delphes=ON" >> Cards/delphes_card.dat
$MG5DIR/bin/mg5_aMC << 'EOF'
launch
EOF


### Example: Loop-Induced gg→ZZ

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate g g > Z Z [noborn=QCD]
output example_ggZZ_loop
launch
EOF


### Example: NLO WW Production

In [ ]:
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > w+ w- [QCD]
output example_WW_NLO
launch
EOF


### Example: EFT Parameter Scan

In [ ]:
%%bash
# Scan MW
for mw in 80 85 90; do
  $MG5DIR/bin/mg5_aMC << 'EOF'
set MW ${mw}
generate p p > e+ e-
output scan_MW_${mw}
launch
EOF
done


### Example: Reweight Scale/PDF Variations

In [ ]:
%%bash
mkdir -p example_dilepton && cd example_dilepton
# In run_card.dat: ensure reweight_scale and reweight_PDF are true
sed -i 's/# reweight_scale/true/' Cards/run_card.dat
sed -i 's/# reweight_PDF/true/' Cards/run_card.dat
$MG5DIR/bin/mg5_aMC << 'EOF'
launch
EOF


### Example: Define Custom Proton PDF

In [ ]:
%%bash
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
define p = u d c s u~ d~ c~ s~
set lhapdf_pdf_set NNPDF31_lo_as_0130
generate p p > j j
output example_customPDF
launch
EOF


### Example: Decay Exclusion Syntax

In [ ]:
%%bash
%%bash
$MG5DIR/bin/mg5_aMC << 'EOF'
generate p p > w+ w- / t
output example_noTopContribution
launch
EOF


### Example: Custom Delphes Card

In [ ]:
%%bash
# Run DelphesHepMC with a custom card externally
$MG5DIR/Delphes/DelphesHepMC cards/delphes_card_CMS.tcl input.hepmc output.root


## Analysis of Delphes Output (Standalone)

In [ ]:
import uproot, awkward as ak, matplotlib.pyplot as plt
file = uproot.open("example_ttbar_dilepton/Events/run_01/tag_1_delphes_events.root")
tree = file["Delphes"]
# plot jet pT histogram
jets = ak.to_numpy(ak.flatten(tree["Jet.PT"].array()))
plt.hist(jets, bins=50)
plt.title("Standalone Jet pT")
plt.show()